# Lab 2: Resistor networks

## Structure of Function Code

We will use the formulas $$A^TCAu = f-A^TCb$$ $$W=C(b-Au)$$ 
that we derived in class. We will compute and create each matrix at a time and then apply the formula, dealing with special cases as they arise (such as singular matrices or edges with infinite current). The code for the function to solve the resistor network is given below, along with descriptive comments on the structure and rationale for each line of code.

In [22]:
## Program that solves for currents and voltages in a resistor network, given the components of the network, i.e. 
## node pairs, voltage and current sources, resistances, and ground node.
import numpy as np

def solve_resistor_network(N, b, f, g, R):
    A = np.zeros((len(b), len(f))) #For our incidence matrix, I find it easier to start with a matrix full of zeroes
    for i in range(len(N)): #This is saying that, "for every row of the matrix A..."
        for j in range(2): #...and "for every entry of each node pair" (there's 2 entries, since it is a pair)
            if j == 0: #..."if the node is listed first"
                A[i][N[i][j]] = -1 #..."then there must be a directed edge leaving the node" (in the direction of current flow)
            if j == 1: #... and "if the node is listed second"
                A[i][N[i][j]] = 1 #..."then there must be a directed edge coming into the node"
    
    A_t = np.transpose(A) #Once we have A, get A transposed
    
    c = [] #Now, we create the C matrix
    no_res = [] #But since we will be dealing with reciprocals, there's a special case we need to watch out for
    for i in range(len(R)):
        if R[i] == 0: #If an edge has no resistor, a resistance of zero would give us an infinite conductance
            c.append(0) #so we just zero it out (so we can carry on with out our calculations) and make it infinite at the end
            no_res.append(i)
        else: #all the resistances are inverted to get the reciprocals
            c.append(1.0/R[i])
    C = np.diag(c) #and we put the reciprocals of the resistances on the diagonal of a matrix
    
    A_tCA = np.matmul(A_t,np.matmul(C,A))
    A_tCb = np.matmul(A_t,np.matmul(C,b))
    
    if np.linalg.det(A_tCA) == 0: #We acccount for the case where the matrix is not invertible
        return "Matrix A_tCA has determinant zero, so it cannot be inverted to obtain a solution with this formula"
    U = np.matmul(np.linalg.inv(A_tCA),A_tCb-f) #We use the formula derived in class to get the nodal voltages
    
    U[g] = 0 #We ground our network
    W = np.matmul(C,b-np.matmul(A,U)) #We use the formula derived in class to get the edge currents
    
    for i in range(len(no_res)): #Finally, all the zero resistance edges have infinite current (short circuit)
        W[no_res[i]] = float('inf')
        
    Solution = [U,W] #Python functions can't return two arguments, so we put them in an array 
    
    return Solution

#### NOTE: I wrote my function with one more extra input: A list of the resistances in the circuit. I was not able to find a way to solve for the voltages and currents without having the resistances in the input. Ohm's Law says that the resistance is equal to the quotient of the voltage drop through the component and the current through it, but since we are trying to find both the voltages and the currents, we can't also have the resistances as unknowns since it is not possible to derive them from other equations either. Also, without this extra input in the function, we are not really using the resistances given to us in Circuits 1, 2, and 3.




## Circuit 1

![Circuit 1](c1.PNG "Circuit 1")

### INPUT
### The edges are given by (1,0), (1,0), (2,1), (2,0), and (0,2), in the direction of current flow.
### The voltage sources are given by -40, 0, 0, 0, and 0.
### The current sources are given by -1, 0, and 1.
### The grounded node is Node 0.
### The resistances are given by 2, 10, 9, 4, and 8.

In [19]:
solve_resistor_network([(1,0),(1,0),(2,1),(2,0),(0,2)],[-40,0,0,0,0],[-1,0,1],0,[2,10,9,4,8])

[array([  0.,  21.,   3.]), array([-9.5  ,  2.1  , -2.   ,  0.75 , -0.375])]

### OUTPUT
### The node voltages are given by 0, 21, and 3.
### The edge currents are given by 9.5, 2.1, 2, 0.75, and 0.375.

## Circuit 2

![Circuit 2](c2.PNG "Circuit 2")

### INPUT
### The edges are given by (1,0), (0,1), and (0,1), in the direction of current flow.
### The voltage sources are given by -30, 0, and 0.
### The current sources are given by -2 and 2.
### The grounded node is Node 0.
### The resistances are given by 5, 3, and 10.

In [26]:
solve_resistor_network([(1,0),(0,1),(0,1)],[-30,0,0],[-2,2],0,[5,3,10])

'Matrix A_tCA has determinant zero, so it cannot be inverted to obtain a solution with this formula'

### OUTPUT
### None. This method cannot be used when the matrix $A^TCA$ is not invertible.

## Circuit 3

![Circuit 3](c3.PNG "Circuit 3")

### INPUT
### The edges are given by (1,0), (2,1), (0,2), and (0,2), in the direction of current flow.
### The voltage sources are given by -30, 0, 0, and -10.
### The current sources are given by 0, -1 and 1.
### The grounded node is Node 0.
### The resistances are given by 0, 50, 30, and 10.

In [21]:
solve_resistor_network([(1,0),(2,1),(0,2),(0,2)],[-30,0,0,-10],[0,-1,1],0,[0,50,30,10])

[array([  0.,  32.,   0.]), array([  inf, -0.64,  0.  , -1.  ])]

### OUTPUT
### The node voltages are given by 0, 32, and 0.
### The edge currents are given by inf, 0.64, 0, and 1.

## When does $\mathrm{solve\_network\_resistor}$ fail?

![Fail Case](failcase.png "Fail Case")

### As we have seen in Circuit 2, the code fails to give us an answer when the matrix $A^TCA$ is not invertible, but that is not a shortcoming of our code but rather of the method. An example where our code would fail would be if we had multiple resistors in one edge, as in the modification of Circuit 3 that we have above (say R4 is 40 ohms). This is because the size of matrix $C$ does not coincide with the number of edges in the circuit, because there is an edge with more than one resistor. This is an easy case to fix by adding a few lines of codes to add up resistors in series so we end up with only one resistor at most per edge.

In [28]:
solve_resistor_network([(1,0),(2,1),(0,2),(0,2)],[-30,0,0,-10],[0,-1,1],0,[0,50,30,10,40])

ValueError: shapes (5,5) and (4,3) not aligned: 5 (dim 1) != 4 (dim 0)